# Implementation

Author: Jørgen S. Dokken

## Test problem
To solve a test problem, we need to choose the right hand side $f$, the coefficient $q(u)$, and the boundary $u_D$. Previously, we have worked with manufactured solutions that can  be reproduced without approximation errors. This is more difficult in nonlinear problems, and the algebra is more tedious. However, we will utilize the UFL differentiation capabilities to obtain a manufactured solution.

For this problem, we will choose $q(u) = 1 + u^2$ and define a two dimensional manufactured solution that is linear in $x$ and $y$:

In [1]:
import ufl
import numpy

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

def q(u):
    return 1 + u**2


domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
x = ufl.SpatialCoordinate(domain)
u_ufl = 1 + x[0] + 2 * x[1]
f = - ufl.div(q(u_ufl) * ufl.grad(u_ufl))

Note that since `x` is a 2D vector, the first component (index 0) represents $x$, while the second component (index 1) represents $y$. The resulting function `f` can be directly used in variational formulations in DOLFINx.

As we now have defined our source term and an exact solution, we can create the appropriate function space and boundary conditions.
Note that as we have already defined the exact solution, we only have to convert it to a Python function that can be evaluated in the interpolation function. We do this by employing the Python `eval` and `lambda`-functions.

In [2]:
V = fem.functionspace(domain, ("Lagrange", 1))
def u_exact(x): return eval(str(u_ufl))

In [3]:
u_D = fem.Function(V)
u_D.interpolate(u_exact)
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets))

We are now ready to define the variational formulation. Note that as the problem is nonlinear, we have to replace the `TrialFunction` with a `Function`, which serves as the unknown of our problem.

In [4]:
uh = fem.Function(V)
v = ufl.TestFunction(V)
F = q(uh) * ufl.dot(ufl.grad(uh), ufl.grad(v)) * ufl.dx - f * v * ufl.dx

## Newton's method
The next step is to define the non-linear problem. As it is non-linear we will use [Newtons method](https://en.wikipedia.org/wiki/Newton%27s_method).
For details about how to implement a Newton solver, see [Custom Newton solvers](../chapter4/newton-solver.ipynb).
Newton's method requires methods for evaluating the residual `F` (including application of boundary conditions), as well as a method for computing the Jacobian matrix. DOLFINx provides the function `NonlinearProblem` that implements these methods. In addition to the boundary conditions, you can supply the variational form for the Jacobian (computed if not supplied), and form and jit parameters, see the [JIT parameters section](../chapter4/compiler_parameters.ipynb).

In [5]:
problem = NonlinearProblem(F, uh, bcs=[bc])

Next, we use the DOLFINx Newton solver. We can set the convergence criteria for the solver by changing the absolute tolerance (`atol`), relative tolerance (`rtol`) or the convergence criterion (`residual` or `incremental`).

In [6]:
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True

We can modify the linear solver in each Newton iteration by accessing the underlying `PETSc` object.

In [7]:
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}ksp_rtol"] = 1.0e-8
opts[f"{option_prefix}pc_type"] = "hypre"
opts[f"{option_prefix}pc_hypre_type"] = "boomeramg"
opts[f"{option_prefix}pc_hypre_boomeramg_max_iter"] = 1
opts[f"{option_prefix}pc_hypre_boomeramg_cycle_type"] = "v"
ksp.setFromOptions()

We are now ready to solve the non-linear problem. We assert that the solver has converged and print the number of iterations.

In [8]:
log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(uh)
assert (converged)
print(f"Number of interations: {n:d}")

2024-08-12 13:22:43.784 (   0.906s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-08-12 13:22:44.179 (   1.301s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-08-12 13:22:44.487 (   1.609s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1057.87 (tol = 1e-10) r (rel) = 1.16853(tol = 1e-06)
2024-08-12 13:22:44.567 (   1.689s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-08-12 13:22:44.854 (   1.977s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 362.964 (tol = 1e-10) r (rel) = 0.400934(tol = 1e-06)
2024-08-12 13:22:44.937 (   2.059s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-08-12 13:22:45.193 (   2.315s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 4:

Number of interations: 8


2024-08-12 13:22:46.313 (   3.436s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-08-12 13:22:46.593 (   3.715s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 8: r (abs) = 5.30788e-06 (tol = 1e-10) r (rel) = 5.86315e-09(tol = 1e-06)
2024-08-12 13:22:46.593 (   3.715s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 8 iterations and 38 linear solver iterations.


We observe that the solver converges after $8$ iterations.
If we think of the problem in terms of finite differences on a uniform mesh, $\mathcal{P}_1$ elements mimic standard second-order finite differences, which compute the derivative of a linear or quadratic funtion exactly. Here $\nabla u$ is a constant vector, which is multiplied by $1+u^2$, giving a second order polynomial in $x$ and $y$, which the finite difference operator would compute exactly. We can therefore, even with $\mathcal{P}_1$ elements, expect the manufactured solution to be reproduced by the numerical method. However, if we had chosen a nonlinearity, such as $1+u^4$, this would not be the case, and we would need to verify convergence rates.

In [9]:
# Compute L2 error and error at nodes
V_ex = fem.functionspace(domain, ("Lagrange", 2))
u_ex = fem.Function(V_ex)
u_ex.interpolate(u_exact)
error_local = fem.assemble_scalar(fem.form((uh - u_ex)**2 * ufl.dx))
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))
if domain.comm.rank == 0:
    print(f"L2-error: {error_L2:.2e}")

# Compute values at mesh vertices
error_max = domain.comm.allreduce(numpy.max(numpy.abs(uh.x.array - u_D.x.array)), op=MPI.MAX)
if domain.comm.rank == 0:
    print(f"Error_max: {error_max:.2e}")

L2-error: 5.91e-16
Error_max: 4.44e-15


2024-08-12 13:22:46.599 (   3.721s) [main            ]      dofmapbuilder.cpp:166   INFO| Checking required entities per dimension
2024-08-12 13:22:46.599 (   3.721s) [main            ]      dofmapbuilder.cpp:264   INFO| Cell type:0, dofmap:500000x6
2024-08-12 13:22:46.635 (   3.757s) [main            ]      dofmapbuilder.cpp:320   INFO| Global index computation
2024-08-12 13:22:46.642 (   3.764s) [main            ]      dofmapbuilder.cpp:637   INFO| Got 2 index_maps
2024-08-12 13:22:46.648 (   3.770s) [main            ]      dofmapbuilder.cpp:644   INFO| Get global indices
